In [30]:
import os
import numpy as np

import torch
import torchvision

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import matplotlib.pyplot as plt

import torchvision.transforms.functional as TF

%matplotlib inline

In [31]:
# image processing package
from PIL import Image, ImageOps
import numpy as np

### Torch dice score

In [32]:
y_true = torch.ones((5,3,256,256))

In [33]:
y_true.shape

torch.Size([5, 3, 256, 256])

In [6]:
y_true_flatten = y_true.view(-1)

In [9]:
y_true_flatten.shape

torch.Size([983040])

In [15]:
y_pred = torch.randn((5,3,256,256))

In [18]:
y_pred[y_pred<0] = 0

In [21]:
y_pred_flatten = y_pred.view(-1)

In [23]:
intersection = 2 * torch.sum(y_true_flatten * y_pred_flatten)

In [24]:
union = torch.sum(y_true_flatten) + torch.sum(y_pred_flatten)

In [25]:
union

tensor(1375389.7500)

In [27]:
(intersection+1)/(union+1)

tensor(0.5705)

## Model Architecture

In [54]:
class LungSegmentationNet(nn.Module):
    def __init__(self, depth=5, root_filter=64):
        super(LungSegmentationNet, self).__init__()
        self.depth = depth
        self.root_filter = root_filter
        print("depth: {}, root filter: {}".format(self.depth, self.root_filter))
    
    # Down sampling
    def forward(self, x):
        long_connection_store = {}
        input_channels = 1
        print("UP SAMPLING")
        for i in range(self.depth):
            out_channel = 2**i * self.root_filter
            print(out_channel)
            x = F.relu(nn.Conv2d(input_channels, out_channel, 3, padding=1)(x))
            x = F.relu(nn.Conv2d(out_channel, out_channel, 3, padding=1)(x))

            if i<self.depth-1:
                long_connection_store[str(i)] = x
                x = F.max_pool2d(x, kernel_size=2)
            input_channels = out_channel

        print("x.shape: {}".format(x.shape))
        print("DOWN SAMPLING")
        
        for i in range(self.depth - 2, -1, -1):
            out_channel = 2**(i) * self.root_filter
            print("input_channels: {}".format(input_channels))
            print("out_channel: {}".format(out_channel))
            
            # long connection from down sampling path.
            long_connection = long_connection_store[str(i)]
            print("long_connection shape: {}".format(long_connection.shape))
            
            up1 = nn.Upsample(scale_factor=2)(x)
            print("up1 shape: {}".format(up1.shape))
            
            up_conv1 = F.relu(nn.Conv2d(input_channels, out_channel, 3, padding=1)(up1))
            print("upsamplingConv : {}".format(up_conv1.shape))
            
            up_conc = torch.cat((up_conv1, long_connection), dim=1)
            print("up_conc : {}\n".format(up_conc.shape))
            
            #  Convolutions
            up_conv1 = F.relu(nn.Conv2d(input_channels, out_channel, 3, padding=1)(up_conc))
            up_conv2 = F.relu(nn.Conv2d(out_channel, out_channel, 3, padding=1)(up_conv1))

            x = up_conv2
            input_channels = out_channel
        print("FINAL shape: {}".format(x.shape))
        x = F.sigmoid(nn.Conv2d(input_channels, 1, 1, padding=0)(x))
        print("FINAL shape: {}".format(x.shape))
        return x
    


In [55]:
unet = LungSegmentationNet(5, 64)

depth: 5, root filter: 64


In [61]:
list(unet.parameters())

[]

In [56]:
for p in unet.parameters():
    print(p)

In [57]:
batch_size = 5
channels = 1
height = 256
width = 256

In [58]:
image = torch.randn(batch_size, channels, height, width)

In [59]:
image.shape

torch.Size([5, 1, 256, 256])

In [60]:
output = unet(image)

UP SAMPLING
64
128
256
512
1024
x.shape: torch.Size([5, 1024, 16, 16])
DOWN SAMPLING
input_channels: 1024
out_channel: 512
long_connection shape: torch.Size([5, 512, 32, 32])
up1 shape: torch.Size([5, 1024, 32, 32])
upsamplingConv : torch.Size([5, 512, 32, 32])
up_conc : torch.Size([5, 1024, 32, 32])

input_channels: 512
out_channel: 256
long_connection shape: torch.Size([5, 256, 64, 64])
up1 shape: torch.Size([5, 512, 64, 64])
upsamplingConv : torch.Size([5, 256, 64, 64])
up_conc : torch.Size([5, 512, 64, 64])

input_channels: 256
out_channel: 128
long_connection shape: torch.Size([5, 128, 128, 128])
up1 shape: torch.Size([5, 256, 128, 128])
upsamplingConv : torch.Size([5, 128, 128, 128])
up_conc : torch.Size([5, 256, 128, 128])

input_channels: 128
out_channel: 64
long_connection shape: torch.Size([5, 64, 256, 256])
up1 shape: torch.Size([5, 128, 256, 256])
upsamplingConv : torch.Size([5, 64, 256, 256])
up_conc : torch.Size([5, 128, 256, 256])

FINAL shape: torch.Size([5, 64, 256, 25

/Users/212633113/Desktop/GE_HealthCare/Experiments/venv/lib/python3.7/site-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [ ]:
# from torchvision import models
# model = models.vgg16()
# print(model)

# Dataset Generator

In [41]:
# PyTorch is channel first instead of channel last as in Keras
class LungSegmentationDataGen(Dataset):
    def __init__(self, dataset, root_dir, args, transforms=None):
        self.dataset = dataset
        self.root_dir = root_dir
        self.args = args
    
    def __len__(self):
        return len(self.dataset)
    
    def preprocess_image(self, path):
        im = Image.open(path)
        im = ImageOps.grayscale(im)
        im = im.resize((self.args.height, self.args.width))  # parameterize height and width
        img = np.array(im)
        img = img/255.
        return img
    
    def __getitem__(self, idx):
        sample = self.dataset[idx]
        img_name, mask_name = sample.rstrip().split(",")
        
        image = self.preprocess_image(os.path.join(self.root_dir, "images", img_name))
        mask = self.preprocess_image(os.path.join(self.root_dir, "masks", mask_name))
        
        # Use albumentation package for augmenting images and mask with same transformations
        
        return torch.from_numpy(image), torch.from_numpy(mask)
        

In [42]:
def dice_loss(y_true, y_pred):
    smooth = 1
    y_true_flatten = y_true.view(-1)
    y_pred_flatten = y_pred.view(-1)
    intersection = 2 * torch.sum(y_true_flatten * y_pred_flatten)
    union = torch.sum(y_true_flatten) + torch.sum(y_pred_flatten)
    dice_score = (intersection + smooth)/(union + smooth)
    return dice_score

In [43]:
def dice_loss(y_true, y_pred):
    return 1 - dice_score(y_true, y_pred)

In [48]:
def train(args):
    dataset = open("dataset.csv", "r").readlines()
    train_set = dataset[:600]
    val_set = dataset[600:]
    root_dir = root_dir = "data/Lung_Segmentation/"

    train_data = LungSegmentationDataGen(train_set, root_dir, args)
    val_data = LungSegmentationDataGen(val_set, root_dir, args)

    train_dataloader = DataLoader(train_data, batch_size=5, shuffle=True, num_workers=4)

    val_dataloader = DataLoader(val_data, batch_size=5, shuffle=True, num_workers=4)
    
    dataloaders = {
        "train": train_dataloader,
        "val": val_dataloader
    }

    dataset_sizes = {"train": len(train_set), "val": len(val_set)}

    print("dataset_sizes: {}".format(dataset_sizes))

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
    model = LungSegmentationNet()
    print("MODEL: {}".format(model))
    model = model.to(device)
    
    # define custom loss function
    criterion = dice_loss
    
    optimizer = optim.Adam(model.parameters())
    
    loss_train = []
    loss_valid = []
    step = 0
    
    for epoch in range(args.epochs):
        print('Epoch {}/{}'.format(epoch, args.epochs - 1))
        print('-' * 10)
        
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for i, data in enumerate(dataloaders[phase]):
                if phase == "train":
                    step += 1
                inputs, y_true = data
                inputs = inputs.to(device)
                y_true = y_true.to(device)
                
                # zero the parameter gradients
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):
                    y_pred = model(inputs)
                    loss = dice_loss(y_true, y_pred)

                    print("LOSS: {}".format(loss))

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss_train.append(loss.item())
                        loss.backward()
                        optimizer.step()
                    if phase == "valid":
                        loss_valid.append(loss.item())
                if phase == "train" and (step + 1) % 10 == 0:
                    print("step:{}, train_loss: {}".format(step, np.mean(loss_train)))
                    loss_train = []
            if phase == "valid":
                print("step:{}, val_loss: {}".format(step, np.mean(loss_valid)))
                loss_valid = []
                

In [49]:
args = args= {
    "batch_size": 2,
    "model_depth": 5,
    "width": 256,
    "root_filter_size": 32,
    "epochs": 100,
    "height": 256
}

In [50]:
train(args)

dataset_sizes: {'train': 600, 'val': 104}
depth: 5, root filter: 64
MODEL: LungSegmentationNet()


ValueError: optimizer got an empty parameter list

## Reading data 

In [ ]:
import os
root_dir = "data/Lung_Segmentation/"

In [ ]:
# MCUCXR_0001_0.png
# CHNCXR_0001_0.png --- CHNCXR_0001_0_mask.png

In [ ]:
images_list = os.listdir(os.path.join(root_dir, "images"))
masks_list = os.listdir(os.path.join(root_dir, "masks"))

In [ ]:
f = open("dataset.csv", "w")
count = 0
for img in images_list:
    img_name = img.split(".png")[0]
    if img.startswith("CHNCXR"):
        mask_name = "{}_mask.png".format(img_name)
    elif img.startswith("MCUCXR"):
        mask_name = img
    if os.path.exists(os.path.join(root_dir, "masks", mask_name)):
        f.write("{}, {}\n".format(img, mask_name))
        count +=1
    
f.close()      

In [ ]:
dataset = open("dataset.csv", "r").readlines()

In [ ]:
len(dataset)